In [1]:
from IPython import __version__ as ipython_version
from pandas import __version__ as pandas_version
from bokeh import __version__ as bokeh_version
from tables import __version__ as tables_version
print(f'IPython - {ipython_version}')
print(f'Pandas - {pandas_version}')
print(f'Bokeh - {bokeh_version}')
print(f'PyTables - {tables_version}')

IPython - 6.3.1
Pandas - 0.22.0
Bokeh - 0.12.14
PyTables - 3.4.3


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [3]:
import local_db
connection = local_db.connection()

In [4]:
today = datetime.now().strftime('%Y%m%d')
print(today)

20180503


In [5]:
begin_year = '2009'

<H2>Stage Ranking codes</H2>

In [6]:
sql_str = "SELECT STAGERANKING_ID, field_name, field_value " + \
          "FROM VWSSTAGERANKING WHERE " + \
          "status = 'A' "
stgrnk = pd.read_sql_query(sql_str, connection)

In [ ]:
# read VWSTAGERANKING.csv and drop unused fields
#stgrnk = pd.read_csv('VWSTAGERANKING.csv')
#stgrnk.drop(['code_table', 'MEDIUM_DESC',
#             'Converted/Confirmed/Accepted/Require SepDate'],
#            inplace=True, axis=1)

In [ ]:
print('stgrnk', stgrnk.shape)

In [ ]:
print('stgrnk\n', stgrnk.dtypes)

In [ ]:
print(stgrnk.columns)

<H2>Stage History data</H2>

In [7]:
sql_str = "SELECT PEOPLE_CODE_ID, ACADEMIC_YEAR, ACADEMIC_TERM, " + \
          "ACADEMIC_SESSION, FIELD_ID, FIELD_DATE " + \
          "FROM STAGEHISTORY WHERE " + \
          f"ACADEMIC_YEAR >= '{begin_year}' "
stg_hist = pd.read_sql_query(sql_str, connection)

In [ ]:
# read STAGEHISTORY.csv
#stg_hist_dtype = {'PEOPLE_CODE_ID': str, 'ACADEMIC_YEAR': str,
#                  'ACADEMIC_TERM': str, 'ACADEMIC_SESSION': str,
#                  'FIELD_ID': np.int64}
#date_cols = ['FIELD_DATE']
#stg_hist = pd.read_csv('STAGEHISTORY.csv', dtype=stg_hist_dtype,
#                       parse_dates=date_cols,
#                       usecols=['PEOPLE_CODE_ID', 'ACADEMIC_YEAR',
#                                'ACADEMIC_TERM', 'ACADEMIC_SESSION',
#                                'FIELD_ID', 'FIELD_DATE'])

In [ ]:
print('stg_hist', stg_hist.shape)

In [ ]:
print('stg_hist')
print(stg_hist.dtypes)

In [8]:
stg_hist = stg_hist.rename(columns={'FIELD_DATE': 'create_date'})

In [9]:
stage_data = pd.merge(stg_hist, stgrnk, left_on=['FIELD_ID'],
                      right_on=['STAGERANKING_ID'], how='left')

In [10]:
keep_fields = ['PEOPLE_CODE_ID', 'ACADEMIC_YEAR', 'ACADEMIC_TERM',
               'ACADEMIC_SESSION', 'field_name', 'field_value', 'create_date']
stage_data = stage_data.loc[~stage_data['create_date'].isnull(), keep_fields]

In [ ]:
print('stage_data', stage_data.shape)

In [ ]:
print('stage_data')
print(stage_data.dtypes)
stage_data.head()

<H2>Academic Data</H2>

In [11]:
sql_str = "SELECT PEOPLE_CODE_ID, ACADEMIC_YEAR, ACADEMIC_TERM, " + \
          "ACADEMIC_SESSION, POPULATION, INQUIRY_FLAG, " + \
          "APPLICATION_FLAG, APPLICATION_DATE, " + \
          "APP_STATUS, APP_STATUS_DATE, " +\
          "APP_DECISION, APP_DECISION_DATE " + \
          "FROM ACADEMIC WHERE " + \
          f"ACADEMIC_YEAR >= '{begin_year}' "
academic = pd.read_sql_query(sql_str, connection)

In [ ]:
academic_dtype = {'PEOPLE_CODE_ID': str, 'ACADEMIC_YEAR': str,
                  'ACADEMIC_TERM': str, 'ACADEMIC_SESSION': str,
                  'APPLICATION_FLAG': str, 'APP_STATUS': str}
date_cols = ['APPLICATION_DATE', 'APP_STATUS_DATE', 'APP_DECISION_DATE']
academic = pd.read_csv('ACADEMIC.csv', dtype=academic_dtype,
                       parse_dates=date_cols,
                       usecols=['PEOPLE_CODE_ID',
                                'ACADEMIC_YEAR', 'ACADEMIC_TERM',
                                'ACADEMIC_SESSION', 'POPULATION',
                                'INQUIRY_FLAG',
                                'APPLICATION_FLAG', 'APPLICATION_DATE',
                                'APP_STATUS', 'APP_STATUS_DATE',
                                'APP_DECISION', 'APP_DECISION_DATE'])

In [ ]:
print('academic', academic.shape)
print('academic')
print(academic.dtypes)
academic.info()

In [ ]:
print(academic['POPULATION'].value_counts().sort_index())
print()
print(academic['POPULATION'].value_counts().sum())

In [12]:
app_data = (academic.loc[~(academic['POPULATION'].isin(['ADVSTU', 'NOND'])) &
                         ((academic['INQUIRY_FLAG'] == 'Y') |
                          (academic['APPLICATION_FLAG'] == 'Y'))]
            )

print('app_data', app_data.shape)
print('app_data')
print(app_data.dtypes)

app_data (32454, 12)
app_data
PEOPLE_CODE_ID               object
ACADEMIC_YEAR                object
ACADEMIC_TERM                object
ACADEMIC_SESSION             object
POPULATION                   object
INQUIRY_FLAG                 object
APPLICATION_FLAG             object
APPLICATION_DATE             object
APP_STATUS                   object
APP_STATUS_DATE      datetime64[ns]
APP_DECISION                 object
APP_DECISION_DATE    datetime64[ns]
dtype: object


In [ ]:
app_data.info()

In [13]:
applied = (app_data[app_data['APP_STATUS'].notnull()]
           .rename(columns={'APP_STATUS': 'field_value',
                            'APP_STATUS_DATE': 'create_date'})
           )
applied.loc[:, 'field_name'] = 'Application Status'
applied = applied.loc[~applied['create_date'].isnull(), keep_fields]

print('applied', applied.shape)
print('applied')
print(applied.dtypes)

applied (13626, 7)
applied
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [ ]:
print(applied['field_value'].value_counts().sort_index())
print(applied['field_value'].value_counts().sum())

In [ ]:
applied.info()

In [14]:
accepted = (app_data[app_data['APP_DECISION'].notnull()]
            .rename(columns={'APP_DECISION': 'field_value',
                             'APP_DECISION_DATE': 'create_date'})
            )
accepted.loc[:, 'field_name'] = 'Application Decision'
accepted = accepted.loc[~accepted['create_date'].isnull(), keep_fields]


print('accepted', accepted.shape)
print('accepted')
print(accepted.dtypes)

accepted (13616, 7)
accepted
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [ ]:
print(accepted['field_value'].value_counts().sort_index())
print(accepted['field_value'].value_counts().sum())

In [ ]:
accepted.info()

In [15]:
# stack Stage History, Academic Applied and Academic Accepted
adm_df = stage_data.append(applied).append(accepted)

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (185700, 7)
adm_df
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [ ]:
adm_df.head()

In [ ]:
adm_df.info()

In [16]:
adm_df = (adm_df.loc[((adm_df['ACADEMIC_TERM'].isin(['FALL', 'SPRING'])) &
                      (adm_df['ACADEMIC_SESSION'] == 'MAIN') &
                      (adm_df['ACADEMIC_YEAR'] >= '2009')
                      )
                     ]
          )

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (123414, 7)
adm_df
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [18]:
# create new fields
adm_df['year_term'] = (adm_df['ACADEMIC_YEAR'] + '.' +
                       adm_df['ACADEMIC_TERM'].str.title())

In [17]:
week_number = (lambda r: (r['create_date'].date().isocalendar()[1])
               if (r['create_date'].date() >= 
                   date((int(r['ACADEMIC_YEAR'])-1), 9, 1))
               else (date((int(r['ACADEMIC_YEAR'])-1), 9, 1)
                     .isocalendar()[1])
               )
adm_df['Week_Number'] = adm_df.apply(week_number, axis=1)

In [ ]:
adm_df[(adm_df['year_term'] == '2018.Fall') & (adm_df['PEOPLE_CODE_ID'] == 'P000054428')]

In [19]:
# convert ACADEMIC_YEAR to numeric keep numeric-valued records
adm_df['ACADEMIC_YEAR'] = pd.to_numeric(adm_df['ACADEMIC_YEAR'],
                                        errors='coerce', downcast='integer')
adm_df = adm_df.loc[adm_df['ACADEMIC_YEAR'].notnull()]

In [20]:
adm_week_number = (lambda r: (r['Week_Number'] -
                              (date((int(r['ACADEMIC_YEAR'])-1), 9, 1)
                              .isocalendar()[1])
                              )
                   if (r['Week_Number'] >= (date((int(r['ACADEMIC_YEAR'])-1), 9, 1)
                                           .isocalendar()[1]))
                   else (53 + r['Week_Number'] -
                         (date((int(r['ACADEMIC_YEAR'])-1), 9, 1)
                         .isocalendar()[1])
                         )
                   )


In [21]:
adm_df['Admissions_Week'] = adm_df.apply(adm_week_number, axis=1)

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (123414, 10)
adm_df
PEOPLE_CODE_ID              object
ACADEMIC_YEAR                int16
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
Week_Number                  int64
year_term                   object
Admissions_Week              int64
dtype: object


In [ ]:
adm_df[(adm_df['year_term'] == '2018.Fall') & (adm_df['PEOPLE_CODE_ID'] == 'P000054428')]

In [ ]:
adm_df.info()

In [ ]:
adm_df.head()

In [ ]:
adm_df.columns

In [ ]:
adm_df[(adm_df['year_term']=='2014.Fall') & (adm_df['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [ ]:
print(adm_df['field_value'].value_counts().sort_index())
print(adm_df['field_value'].value_counts().sum())

In [22]:
adm_keep_values = ['300', 'ACC', 'ACXL', 'CANC', 'DEF', 'DEFR', 'DENY', 'DPAC',
                   'TRDP', 'TRPD', 'TRNS', 'WAIT', '500', 'PEND', 'COMP']
adm_keep_cols = ['PEOPLE_CODE_ID', 'year_term', 'Admissions_Week',
                 'field_value']
adm_df = adm_df.loc[(adm_df['field_value'].isin(adm_keep_values)),
                    adm_keep_cols]

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (58796, 4)
adm_df
PEOPLE_CODE_ID     object
year_term          object
Admissions_Week     int64
field_value        object
dtype: object


In [ ]:
adm_df.head()

In [ ]:
adm_df[(adm_df['year_term']=='2014.Fall') & (adm_df['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [23]:
# admissions status table
admission_status = {'300': 'Applied', 'ACC': 'Accepted', 'ACXL': 'Canceled',
                    'CANC': 'Canceled', 'DEF': 'Canceled', 'DEFR': 'Canceled',
                    'DENY': 'Canceled', 'DPAC': 'Deposited',
                    'TRDP': 'Deposited', 'TRPD': 'Deposited',
                    'TRNS': 'Accepted', 'WAIT': 'Accepted',
                    '500': 'Deposited', 'PEND': 'Applied',
                    'COMP': 'Applied',
                    }
adm_stat = pd.DataFrame(list(admission_status.items()),
                        columns=['field_value', 'admission_status'])

In [24]:
adm_df1 = (pd.merge(adm_df, adm_stat, on=['field_value'], how='left')
           .drop(['field_value'], axis=1)
           .drop_duplicates(['PEOPLE_CODE_ID', 'year_term', 'Admissions_Week',
                             'admission_status'])
           )

print('adm_df1', adm_df1.shape)
print('adm_df1')
print(adm_df1.dtypes)

adm_df1 (29676, 4)
adm_df1
PEOPLE_CODE_ID      object
year_term           object
Admissions_Week      int64
admission_status    object
dtype: object


In [ ]:
adm_df1.head()

In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['admission_status']=='Deposited')].head(30)

In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [25]:
adm_df1 = (adm_df1.sort_values(['year_term', 'PEOPLE_CODE_ID',
                                'admission_status', 'Admissions_Week'])
           .drop_duplicates(['year_term', 'PEOPLE_CODE_ID',
                             'admission_status'],
                            keep='first')
           )

In [ ]:
print('adm_df1', adm_df1.shape)

In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['admission_status']=='Deposited')].head(30)

In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [ ]:
adm_df2 = (adm_df1.loc[(adm_df1['year_term'].isin(['2014.Fall', '2015.Fall']))]
           .reset_index()
           .set_index(['year_term', 'PEOPLE_CODE_ID', 'admission_status'])
           .drop(['index'], axis=1)
           .unstack(level=-1)
          )

print('adm_df2', adm_df2.shape)
print('adm_df2')
print(adm_df2.dtypes)

In [ ]:
adm_df2.head()

In [ ]:
#q = adm_df1.loc[(adm_df1['year_term']=='2014.Fall'), ['PEOPLE_CODE_ID', 'Admissions_Week', 'admission_status']]
q = adm_df1.loc[((adm_df1['year_term']=='2014.Fall') | (adm_df1['year_term']=='2015.Fall')), ['year_term', 'PEOPLE_CODE_ID', 'Admissions_Week', 'admission_status']]

In [ ]:
print(q.shape)
q.head()

In [ ]:
w = q.set_index(['year_term', 'PEOPLE_CODE_ID'])

In [ ]:
print(w.shape)
w.head(30)

In [ ]:
# just 2018.Fall for testing
adm_df1 = adm_df1.loc[(adm_df1['year_term']=='2018.Fall') , ['year_term', 'PEOPLE_CODE_ID', 'Admissions_Week', 'admission_status']]

In [26]:
e = adm_df1.pivot_table(index=['year_term', 'PEOPLE_CODE_ID'],
                        columns=['admission_status'],
                        values=['Admissions_Week']
                        )
e = e.fillna(54)
print(e.shape)
e.head(30)

(11525, 4)


Admissions_Week                           
admission_status                Accepted Applied Canceled Deposited
year_term PEOPLE_CODE_ID                                           
2009.Fall P000000564                25.0    54.0     54.0      25.0
          P000006539                37.0    54.0     54.0      37.0
          P000022053                29.0    54.0     54.0      29.0
          P000022130                28.0    54.0     54.0      28.0
          P000022259                 0.0    54.0     54.0       0.0
          P000022701                50.0    50.0     54.0      50.0
          P000022722                 3.0     3.0     54.0      43.0
          P000022725                54.0    49.0     49.0      54.0
          P000022727                 5.0    54.0     54.0       5.0
          P000022729                 6.0    54.0     54.0       6.0
          P000022730                 6.0    54.0     54.0       6.0
          P000022734                 7.0    54.0     54.0       7.0
          P000022736                 7.0    54.0     54.0       7.0
          P000022758                 9.0    54.0     54.0       9.0
          P000022759                 9.0    54.0     54.0       9.0
          P000022761                 9.0    54.0     54.0       9.0
          P000022762                 9.0    54.0     54.0       9.0
          P000022768                10.0    54.0     54.0      10.0
          P000022769                10.0    54.0     54.0      10.0
          P000022774                11.0    54.0     41.0      11.0
          P000022776                11.0    54.0     54.0      11.0
          P000022782                11.0    54.0     54.0      11.0
          P000022783                11.0    54.0     54.0      11.0
          P000022785                11.0    54.0     54.0      11.0
          P000022787                12.0    54.0     54.0      12.0
          P000022788                12.0    54.0     54.0      12.0
          P000022796                13.0    54.0     54.0      13.0
          P000022797                13.0    54.0     54.0      13.0
          P000022798                13.0    54.0     54.0      13.0
          P000022799                13.0    54.0     54.0      13.0

In [ ]:
e.to_csv('2018_Fall_pivot.csv')

In [27]:
# function returns status for week
def f_status(field, data_frame, n):
    f_week = (lambda df: 1
              if ((df[('Admissions_Week', field)] <= n) &
                  (df[('Admissions_Week', 'Canceled')] > n))
              else 0
              )
    return data_frame.apply(f_week, axis=1)

In [ ]:
e[('Admissions_Week', 'Canceled')]

In [ ]:
f_status('Applied', e, 1).to_csv('2018_Fall_w1.csv')

In [28]:
# function returns DataFrame of 53 week status values
def fill_weeks(field, data_frame):
    weeks = range(0, 54)
    r = pd.DataFrame(np.zeros((data_frame.shape[0], 54)),
                     index=data_frame.index,
                     columns=[f'{w:02d}' for w in weeks])
    for w in weeks:
        f = f'{w:02d}'
        r.loc[:, f] = f_status(field, data_frame, w)
        r.loc[:, 'stage'] = field

    r = r.reset_index().set_index(['year_term', 'stage', 'PEOPLE_CODE_ID'])
   
    return r

In [29]:
stage_list = ['Applied', 'Accepted', 'Deposited']
w = pd.DataFrame()
for stg in stage_list:
    w = pd.concat([w, fill_weeks(stg, e)])


In [ ]:
w.to_csv('2018_Fall_stages.csv')

In [ ]:
w.head()


In [ ]:

print(w.shape)
w.head(30)

In [ ]:
w

In [ ]:
w.info()

In [30]:
# add CURRICULUM field
sql_str = "SELECT PEOPLE_CODE_ID, ACADEMIC_YEAR, ACADEMIC_TERM, " + \
          "ACADEMIC_SESSION, CURRICULUM, PRIMARY_FLAG " + \
          "FROM ACADEMIC WHERE " + \
          "PRIMARY_FLAG = 'Y' AND " + \
          f"ACADEMIC_YEAR >= '{begin_year}' "
curriculum_df = pd.read_sql_query(sql_str, connection)

In [ ]:
print(curriculum_df.shape)
print(curriculum_df.head())

In [31]:
curriculum_df['year_term'] = (curriculum_df['ACADEMIC_YEAR'] + '.' +
                              curriculum_df['ACADEMIC_TERM'].str.title())
curriculum_df = curriculum_df.rename(columns={'CURRICULUM': 'curriculum'})


In [ ]:
print(curriculum_df.shape)
print(curriculum_df.head())

In [32]:
curr_flds = ['PEOPLE_CODE_ID', 'year_term', 'curriculum']
curriculum_df = curriculum_df[curr_flds]
curriculum_df = curriculum_df.drop_duplicates(curr_flds)

In [ ]:
print(curriculum_df.shape)
print(curriculum_df.head())

In [ ]:
print(w.head())

In [33]:
y = pd.merge(w.reset_index(), curriculum_df,
             on=['year_term', 'PEOPLE_CODE_ID'], how='left')

In [ ]:
print(y.shape)

In [ ]:
print(y.head())

In [ ]:
y.head()

In [34]:
y.to_hdf('data/stage_data', key='weekly', mode='w', data_columns=True, complevel=0)

C:\Anaconda3\envs\py36\lib\site-packages\pandas\core\generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['year_term', 'stage', 'PEOPLE_CODE_ID', 'curriculum']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [35]:
df = pd.read_hdf('data/stage_data', key='weekly')

In [36]:
summ = df.groupby(['year_term', 'stage']).sum()


In [ ]:
summ.loc[('2018.Fall', )]

In [ ]:
(summ.loc[('2013.Fall', )].transpose()).to_csv('2013.Fall.csv')
(summ.loc[('2014.Fall', )].transpose()).to_csv('2014.Fall.csv')
(summ.loc[('2015.Fall', )].transpose()).to_csv('2015.Fall.csv')
(summ.loc[('2016.Fall', )].transpose()).to_csv('2016.Fall.csv')

In [57]:
(summ.loc[('2018.Fall', )].transpose()).to_csv('2018.Fall.csv')


In [ ]:
writer = pd.ExcelWriter(f'admissions_historic_data_{today}.xlsx')
terms = ['2010.Fall', '2011.Fall', '2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall', '2018.Fall', ]
#terms = ['2018.Fall', ]
for t in terms:
    (summ.loc[(t, )].transpose()).to_excel(writer, t)
writer.save()

In [37]:
summ_t = summ.transpose()
summ_t.head()

year_term 2009.Fall                   2009.Spring                   2010.Fall  \
stage      Accepted Applied Deposited    Accepted Applied Deposited  Accepted   
00                2       0         2           0       0         0         3   
01                2       0         2           0       0         0         4   
02                2       0         2           0       0         0         6   
03                3       4         2           1       0         1         7   
04                3       4         2           1       0         1         8   

year_term                   2010.Spring    ...    2017.Fall 2017.Spring  \
stage     Applied Deposited    Accepted    ...    Deposited    Accepted   
00              0         3           2    ...           10           8   
01              0         4           2    ...           10           8   
02              0         6           2    ...           11           8   
03              0         7           2    ...           11           8   
04              0         8           2    ...           11          10   

year_term                   2018.Fall                   2018.Spring          \
stage     Applied Deposited  Accepted Applied Deposited    Accepted Applied   
00             12         4         7      26         3           2       4   
01             12         4         8      32         3           2       4   
02             12         4         8      34         3           2       5   
03             13         4         8      42         3           2       6   
04             14         4         8      50         3           2       8   

year_term            
stage     Deposited  
00                3  
01                3  
02                3  
03                3  
04                3  

[5 rows x 60 columns]

In [ ]:
summ_t.index

In [ ]:
summ_t[('2018.Fall', 'Deposited')]

In [38]:
terms = ['2010.Fall', '2011.Fall', '2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall', '2018.Fall', ]
this_term = '2018.Fall'
#print(terms)
#print(this_term)
terms.remove(this_term)

#print(terms)

In [40]:
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Blues8

#df = 

p = figure(plot_width=800, plot_height=600, title="Weekly Deposits",
           x_axis_label="Week Number", y_axis_label="Deposits",
           tools="pan,wheel_zoom,box_zoom,save,reset",
          )

c=7
for t in terms:
    p.line(summ_t.index, summ_t[(t, 'Deposited')], color=Blues8[c], legend=t)
    c -= 1

p.line(summ_t.index, summ_t[(this_term, 'Deposited')], color='red', legend=this_term)

p.legend.location = "top_left"

output_file("weekly_deposits.html")

show(p)

In [1]:
import pandas as pd
from datetime import date

from bokeh.layouts import widgetbox, row
from bokeh.models.widgets import RadioGroup
#from bokeh.plotting import figure, output_file, show
from bokeh.plotting import figure, curdoc
from bokeh.palettes import Blues8

In [2]:
today = date.today()
today_str = today.strftime('%Y%m%d')

In [3]:
def adm_week(d):
    """
    returns calendar week number and Admissions Week Number for a given date, d
    """
    year = d.year
    if d >= date(year, 9, 1):
        adm_year_start = year
    else:
        adm_year_start = year - 1
    
    week_number = d.isocalendar()[1]
    adm_start_week = date(adm_year_start, 9, 1).isocalendar()[1]

    if week_number >= adm_start_week:
        adm_week_number = week_number - adm_start_week
    else:
        adm_week_number = 53 + (week_number - adm_start_week)
    
    return (week_number, adm_week_number)


In [4]:
df = pd.read_hdf('data/stage_data', key='weekly')

print(df.shape)
print(df.head())

(34575, 58)
   year_term    stage PEOPLE_CODE_ID  00  01  02  03  04  05  06     ...      \
0  2009.Fall  Applied     P000000564   0   0   0   0   0   0   0     ...       
1  2009.Fall  Applied     P000006539   0   0   0   0   0   0   0     ...       
2  2009.Fall  Applied     P000022053   0   0   0   0   0   0   0     ...       
3  2009.Fall  Applied     P000022130   0   0   0   0   0   0   0     ...       
4  2009.Fall  Applied     P000022259   0   0   0   0   0   0   0     ...       

   45  46  47  48  49  50  51  52  53  curriculum  
0   0   0   0   0   0   0   0   0   0        FIFO  
1   0   0   0   0   0   0   0   0   0        NRMP  
2   0   0   0   0   0   0   0   0   0        FWSW  
3   0   0   0   0   0   0   0   0   0        FIFO  
4   0   0   0   0   0   0   0   0   0        FWSW  

[5 rows x 58 columns]


In [8]:
all_terms = sorted(list(df['year_term'].dropna().unique()))
all_terms = [l for l in all_terms if 'Fall' in l]

In [9]:
print(all_terms)

['2009.Fall', '2010.Fall', '2011.Fall', '2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall', '2018.Fall']


In [45]:
terms = ['2010.Fall', '2011.Fall', '2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall', '2018.Fall', ]
this_term = '2018.Fall'
terms.remove(this_term)


In [46]:
all_terms[-1]

'2018.Spring'

In [146]:
summ = df.groupby(['year_term', 'stage']).sum()
summ_t = summ.transpose()


In [147]:
week_number, adm_week_number = adm_week(today)

In [148]:
curr_list = sorted(list(df['curriculum'].dropna().unique()))
print(type(curr_list))
print(curr_list)

<class 'list'>
['AALM', 'BAKA', 'BASM', 'BIOG', 'BMES', 'CASM', 'COMM', 'CULA', 'CULB', 'CULO', 'EBSB', 'ECOR', 'ENST', 'ENVS', 'FBIO', 'FEFM', 'FFOP', 'FIFO', 'FORT', 'FRRM', 'FSBM', 'FVEG', 'FWSF', 'FWSW', 'GISC', 'HRMT', 'HRTM', 'INAS', 'INST', 'LAGS', 'NRCM', 'NRMP', 'NRSS', 'PACM', 'PRFM', 'PSYCH', 'RATE', 'RECR', 'SCWL', 'SURV', 'UNDM']


In [149]:
title = f"Admissions Weekly Summary - Week {adm_week_number} ({today_str})"
print(title)

Admissions Weekly Summary - Week 36 (20180501)


In [155]:
def create_figure():
    stage = stage_list[stage_n.active]
    p = figure(plot_width=800, plot_height=600, title=title,
           x_axis_label="Week Number", y_axis_label=stage,
           tools="pan,wheel_zoom,box_zoom,save,reset",
          )
    c=7
    
    for t in terms:
        p.line(df.index, summ_t[(t, stage)], color=Blues8[c], legend=t)
        c -= 1

    p.line(summ_t.index, summ_t[(this_term, stage)], color='red', legend=this_term)

    p.legend.location = "top_left"
    
    return p


    

In [156]:
def update(attr, old, new):
    layout.children[1] = create_figure()

In [157]:
stage_list = ['Applied', 'Accepted', 'Deposited']
stage_n = RadioGroup(labels=stage_list, active=2)
stage_n.on_change('active', update)

In [158]:
controls = widgetbox([stage_n])
layout = row(controls, create_figure())


C:\Anaconda3\envs\py36\lib\site-packages\bokeh\models\sources.py:138: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 34575), ('y', 54)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
C:\Anaconda3\envs\py36\lib\site-packages\bokeh\models\sources.py:138: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 34575), ('y', 54)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
C:\Anaconda3\envs\py36\lib\site-packages\bokeh\models\sources.py:138: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 34575), ('y', 54)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
C:\Anaconda3\envs\py36\lib\site-packages\bokeh\models\sources.py:138: BokehUserWarning: ColumnDataSource's columns must be of t

In [159]:
curdoc().add_root(layout)
curdoc().title = "Admissions Weekly Report"

In [161]:
show(p)

NameError: name 'p' is not defined

In [ ]:
output_file("weekly_admissions.html")

show(p)